In [1]:
from sys import path
from pathlib import Path

import pandas as pd
import numpy as np
pd.options.display.float_format = '{:,.2f}'.format

# Se genera la ruta absoluta al directorio "Comisiones"
path_package = str(Path.cwd().parent)

# Se agrega la ruta al directorio sys.path
path.append(path_package)


from algoritmos.algoritmo_cobranza_mes import cobranza_mes_func

from descriptions.descriptions import descrip_sales_users_df, descrip_product_df, descrip_partner_df

In [2]:
cobranza_enero = cobranza_mes_func(1)
cobranza_febrero = cobranza_mes_func(2)
cobranza_marzo = cobranza_mes_func(3)
cobranza_abril = cobranza_mes_func(4)
cobranza_mayo = cobranza_mes_func(5)

cobranza_año = pd.concat([cobranza_enero, cobranza_febrero, cobranza_marzo, cobranza_abril, cobranza_mayo], ignore_index=True)

In [3]:
from sqlalchemy import create_engine
from pathlib import Path

import pandas as pd
import numpy as np

pd.options.display.float_format = '{:,.2f}'.format

db_file = 'comisiones.db'
db_file_path_str = str(Path().cwd().parent.parent.joinpath(f'data/{db_file}'))

engine = create_engine(f'sqlite:///{db_file_path_str}')

In [27]:
with engine.connect() as conn, conn.begin():  
    ventas_año = pd.read_sql_table('ventas_año', conn, dtype_backend='numpy_nullable')

engine.dispose()

In [45]:
wep = (cobranza_año.merge(
              ventas_año[['fact_doc_id', 'salesperson_id', 'salesperson_name', 'active',
              'sale_team_id', 'sale_team_description', 'business_model', 'warehouse']], how='left', on='fact_doc_id')).merge(
                     descrip_partner_df, how='left', left_on= 'fact_doc_cliente', right_on= 'partner_id'
                     )[[
                     'fact_doc_id', 'fact_doc_name', 'fact_doc_cliente', 'partner_name', 'fact_doc_date', 'fact_doc_total', 
                     'fact_doc_deuda', 'pay_journal', 'ref', 'pay_amount', 'pay_date', 'pay_pos', 'salesperson_id', 'salesperson_name', 
                     'active', 'sale_team_id', 'sale_team_description', 'business_model', 'warehouse']]

wep.columns = ['fact_doc_id', 'fact_doc_name', 'fact_doc_cliente_id', 'fact_doc_cliente_name',
       'fact_doc_date', 'fact_doc_total', 'fact_doc_deuda', 'pay_journal',
       'ref', 'pay_amount', 'pay_date', 'pay_pos', 'salesperson_id',
       'salesperson_name', 'active', 'sale_team_id', 'sale_team_description',
       'business_model', 'warehouse']